In [ ]:
import json
import urllib
import sys
import os
import time
import re
import hashlib

_sha256 = '[a-z0-9]{64}' # sha256에 맞는 정규표현식 정의

path_dir = '/home/user/Desktop/레이블링 테스트'
file_list = os.listdir(path_dir)

sha256_pattern = re.compile(_sha256)

class vtAPI():
    def __init__(self):
        self.api = ''
        self.base = 'https://www.virustotal.com/vtapi/v2/'
        
    # Virustotal에 주어진 악성코드가 검사된 적이 있는지 확인
    def getReport(self,sha256):
        param = {'resource':sha256,'apikey':self.api,'allinfo': '1'}
        url = self.base + "file/report"
        data = urllib.parse.urlencode(param) # Python 3에는 urllib가 나뉘어져 있음 (parse, request)
        data = data.encode('utf-8') # urlopen()에 data를 넣기 위해, 인코딩이 되어 있어야 함.
        result = urllib.request.urlopen(url,data)

        jdata = json.loads(result.read())
        print(jdata)

        if jdata['response_code'] == 0:
            print(sha256 + " -- Not Found in VT")
            return "no"
        else:
            print("=== Results for SHA256: ", jdata['sha256'], "\tDetected by: ", jdata['positives'])
            return jdata['positives']

    # Virustotal에 악성코드 검사를 의뢰함
    def reqScan(self,filepath):
        print("- Requesting a new scan")
        param = {'file':filepath,'apikey':self.api}
        url = self.base + "file/scan"
        data = urllib.parse.urlencode(param) # Python 3에는 urllib가 나뉘어져 있음 (parse, request)
        data = data.encode('utf-8') # urlopen()에 data를 넣기 위해, 인코딩이 되어 있어야 함.
        result = urllib.request.urlopen(url,data)

        jdata = json.loads(result.read())

        return jdata
    
    # sha256해쉬값을 구함
    def getSha256(self, filepath, blocksize=8192):
        sha256 = hashlib.sha256()
        try:
            f = open(filepath, "rb")
        except IOError as e:
            print("file open error", e)
            return
        while True:
            buf = f.read(blocksize)
            if not buf:
                break
            sha256.update(buf)
        return sha256.hexdigest()


def main():
    vt = vtAPI()
    i = 0
    for file in file_list:
        before = path_dir + "/" + file
        name_check = re.search(sha256_pattern, file) # 악성코드 파일명이 sha256형식에 맞는지 확인

        if name_check == None:
            file = vt.getSha256(before)


        i += 1
        rns = vt.getReport(file) #rns는 악성코드가 백신에 탐지된 갯수를 의미함

        if(rns == "no"): # Virustotal에 악성코드 검사 이력이 없을 때
            file_path = os.getcwd() + "/" + file
            rns = vt.reqScan(file_path)
            file = rns['sha256']

            while True: # 이후 검사이력이 생성될때까지 실행
                time.sleep(20)
                rns = vt.getReport(file)
                if(rns != "no"):
                    break

        after = path_dir + "/" + str(rns) + "#" + file # 본 코드를 처리한 악성코드의 이름 변경

        print("Processed " + str(i) + " files - "+ after)
        os.rename(before, after)

        time.sleep(15)

        


if __name__ == '__main__':
    main()